In [1]:
#ライブラリのインポート

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#　requestsメソッドを使って、サイトからHTMLの情報を抽出する。
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTy1ZbSq5OAVbEqScDw_Uf_KfLKydg0uUWBaBEPK_DXaBPCHmwv4ru9MzZHHnVHVmRVvLvOlndJ2ZNZ/pubhtml?gid=159569114&amp;single=true&amp;widget=true&amp;headers=false&amp;gid=0&amp;range=A:F%22'
res = requests.get(url)

In [3]:
#　BeautifulSoup()の設定を行う。
soup = BeautifulSoup(res.text,'html.parser')

In [23]:
# 最新弾の情報を抽出する。
poke_new = soup.find('div',id ='1490875147')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_new.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,349):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)
    
# データの結合を行う。
df_new = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_new.columns = df_new.iloc[0]
df_new = df_new.drop(0)

#価格が欠損値になっている行を削除する。
df_new = df_new.dropna(subset = ['買取'])

#買取データの日付を取得する。
import datetime 
today = datetime.date.today()
today = today.strftime('%y-%m-%d')

df_new.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [24]:
#### 通常の情報を抽出する。
poke_norm = soup.find('div',id ='159569114')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_norm.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,3644):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)

# データの結合を行う。
df_norm = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_norm.columns = df_norm.iloc[0]
df_norm = df_norm.drop(0)

#価格が欠損値になっている行を削除する。
df_norm = df_norm.dropna(subset = ['買取価格'])

#買取データの日付を取得する。
import datetime 
today = datetime.date.today()
today = today.strftime('%y-%m-%d')

df_norm.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [25]:
#更新日のデータを完成させる。
#データを結合する。
df_today = pd.concat([df_norm,df_new],ignore_index = True)
df_today.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [26]:
#集計データを読み込む。
df_preday = pd.read_csv('card_data.csv')

In [27]:
#結合のためのkeyを作る。(df_all)

df_today['key'] = df_today['カード名'].str.cat(df_today['型番'],sep = ' , ',na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['仕様'],sep = ' ,' ,na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['R'],sep = ' ,' ,na_rep='')

print('df_todayのユニークなkeyの個数:',len(df_today['key'].unique()))
print('df_todayのデータの列数:',len(df_today))
print('df_todayのkeyに重複のあるデータ:\n',df_today[df_today['key'].duplicated()])

df_todayのユニークなkeyの個数: 3648
df_todayのデータの列数: 3652
df_todayのkeyに重複のあるデータ:
             カード名    仕様   R       型番   タイプ 21-01-26                         key
731    クラッシュハンマー  None   -  051/072   グッズ       10    クラッシュハンマー , 051/072 , ,-
1841       ピカチュウ  None  UR  056/053     雷   12,000       ピカチュウ , 056/053 , ,UR
2175  ゲンシカイオーガEX  None  RR  032/070     水       50  ゲンシカイオーガEX , 032/070 , ,RR
3052   ポケモンレンジャー  None   U  054/054  サポート       30    ポケモンレンジャー , 054/054 , ,U


In [28]:
#結合のためのkeyを作る。(df_all)

df_preday['key'] = df_preday['カード名'].str.cat(df_preday['型番'],sep = ' , ',na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['仕様'],sep = ' ,' ,na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['R'],sep = ' ,' ,na_rep='')

print('df_predayのユニークなkeyの個数:',len(df_preday['key'].unique()))
print('df_predayのデータの列数:',len(df_preday))
print('df_predayのkeyに重複のあるデータ:\n',df_preday[df_preday['key'].duplicated()])

df_predayのユニークなkeyの個数: 3649
df_predayのデータの列数: 3653
df_predayのkeyに重複のあるデータ:
             カード名   仕様   R       型番   タイプ 21-01-25                         key
722    クラッシュハンマー  NaN   -  051/072   グッズ       10    クラッシュハンマー , 051/072 , ,-
1829       ピカチュウ  NaN  UR  056/053     雷   12,000       ピカチュウ , 056/053 , ,UR
2163  ゲンシカイオーガEX  NaN  RR  032/070     水       50  ゲンシカイオーガEX , 032/070 , ,RR
3042   ポケモンレンジャー  NaN   U  054/054  サポート       30    ポケモンレンジャー , 054/054 , ,U


In [29]:
#keyが被る行にtodayの数値を代入する。

df_preday[today] = 0
df_preday

for i in range(len(df_preday)):
    for j in range(len(df_today)):
        if df_preday.loc[i,'key'] == df_today.loc[j,'key']:
            df_preday.loc[i,today] = df_today.loc[j,today]            

In [30]:
count = 0
for i in range(len(df_today)):
    flg = 0
    for j in range(len(df_preday)):
        if df_today.loc[i,'key'] == df_preday.loc[j,'key']:
            flg = 1
    if flg == 0:
        df_preday = pd.concat([df_preday,df_today[i:i+1]])
        df_preday.reset_index(drop = True,inplace=True)
        count = count + 1

In [32]:
#データの整理を行う。
df_update = df_preday
df_update = df_update.drop('key',axis = 1)

In [37]:
#新規に値段がついたカードを表示する。
print('■ 新規に値段がついたカード')
df_update.tail(count)

■ 新規に値段がついたカード


,カード名,仕様,R,型番,タイプ,21-01-25,21-01-26
3653,マラカッチ,None,C,003/060,草,NaN,10
3654,ガラルマタドガス,R仕様,-,105/190,悪,NaN,10
3655,妖しいカンヅメ,None,U,063/070,グッズ,NaN,10
3656,おうえんYホーン,None,U,089/100,グッズ,NaN,10
3657,トレーニングコート,None,U,069/070,スタジアム,NaN,10
3658,トレーニングコート,None,-,026/033,スタジアム,NaN,10
3659,トレーニングコート,None,-,180/190,スタジアム,NaN,10
3660,トレーニングコート,ミラー,-,180/190,スタジアム,NaN,10
3661,ニドクイン,None,U,041/095,超,NaN,10
3662,イツキ,None,U,044/049,サポート,NaN,10


In [38]:
#買取がなくなったカードを表示する。
print('■ 買取がなくなったカード')
df_update[df_update[today] == 0]

■ 買取がなくなったカード


,カード名,仕様,R,型番,タイプ,21-01-25,21-01-26
3557,MミュウツーEX,X,SR,063/059,超,"3,000",0
3558,MミュウツーEX,Y,SR,063/059,超,"3,000",0
3559,MミュウツーEX,Y,-,052/131,超,150,0
3560,ミュウ,NaN,R,051/087,超,"4,000",0
3561,ミュウ,NaN,R,018/040,超,"4,000",0
3562,ミュウ,NaN,R,027/078,超,100,0
3563,ミュウ,NaN,-,044/171,超,30,0
3564,ミュウ,NaN,-,009/048,超,30,0
3565,ミュウ,NaN,P,220/BW-P,超,"8,000",0
3566,ミュウ,ミラー,-,044/171,超,50,0


In [40]:
df_update.to_csv('card_data.csv',index = None)